# Setup Code for VM related items
## !!!!When Saving, please ping revision so its saved in revision history!!!
Probs need to run anything under this section before code


In [1]:
!pip install alpaca-trade-api

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 61kB 2.9MB/s 


Pandas Technical Analysis (Pandas TA) is an easy to use library that leverages the Pandas library with more than 120 Indicators and Utility functions. 

INFO HERE : https://github.com/twopirllc/pandas-ta#installation

USEFUL EXAMPLE: https://github.com/twopirllc/pandas-ta/blob/master/examples/AIExample.ipynb

In [2]:
!pip install --upgrade ta

  Created wheel for ta: filename=ta-0.7.0-cp37-none-any.whl size=28716 sha256=77711f720d1cd932b9476a300bb3502dab1a0cd6cf5a2e10c04621bf21582486
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [3]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 6.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=7ffbe2fd459f16397467902fb09ba5c0fbb1a409d9f0f840c626053a3019f30d
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


#Code
code cells are unfortunately not linked together and a such all code for program has to go on one cell

Yahoo Finance Key : 75ee96eb2fmsha6dbdb2310728dfp1301d2jsnbf21f7a615c1

In [1]:
import alpaca_trade_api as alpacaapi
import yfinance as yf, pandas as pd, shutil, os, time, glob
import matplotlib.pyplot as plt
import pytz as pytz
import requests
import json
import numpy as np
import datetime
import time
from datetime import timedelta
from ta import add_all_ta_features
from ta.utils import dropna
from ta.momentum import RSIIndicator
from ta.trend import MACD, SMAIndicator, EMAIndicator
from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive


drive.mount("/content/drive/")
api = alpacaapi.REST("PKCR2CYNFRSM1D7OAUM1", "x6YRbTOfAsDSVMs73IsVmLJ1dbUCTTj2s4Md70bO", "https://paper-api.alpaca.markets")

################
# getStockData #
################

# https://alpaca.markets/docs/api-documentation/api-v2/market-data/bars/
# Check the above link for specific usage if you need to use it in your function
def getStockData(tick, length, frame):
  barset = api.get_barset(tick, frame , limit=length).df
  return barset

#Example usage
#  This will get the info for Tesla for the last 100 minutes
#  getStockData("TSLA", 100, "minute")


#####################
# getStockDataMulti #
#####################

# getStockDataMulti creates a list of dataframes containing
# time-aggregated price and volume data 
def getStockDataMulti(ticks, length, frame):
  dataframes = []
  for tick in ticks:
    df = getStockData(tick, length, frame)
    dataframes.append({ tick: df })
  return dataframes

# Example usage
#   stocks = ["TSLA", "AAPL", "SNAP"]
#   data = getStockDataMulti(stocks, 5, "minute")
#   print(data)

######################
# getTrendingTickers #
######################

# getTrendingTickers creates a list of the current top 20
# trending stocks from Yahoo Finance
def getTrendingTickers():
  # build request
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-trending-tickers"
  querystring = {"region":"US"}
  headers = {
      'x-rapidapi-key': "535fe75f64msh88e3286b5a9d502p19834bjsnc6a3659ddec5",
      'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
  }
  # execute request
  response = requests.request("GET", url, headers=headers, params=querystring)
  raw_data = json.loads(response.text)['finance']['result']
  # extract symbols
  symbols = []
  for quote in raw_data[0]['quotes']:
    symbols.append(quote['symbol'])
  return symbols

# Example usage
#   symbols = getTrendingTickers()
#   print(symbols)



#############
# sellLimit #
#############

# A sell limit is submitted as a gtc(Good 'Til Cancelled) time in force which 
# keeps the sell limit order open until it is fulfilled or cancelled.
# You must pass in the current price of the tick as the third argument when the
# sell limit is placed. This is so the 10% price increase can be calculated.
def sellLimit(tick, quantity, current_price):
    try:
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='sell',
            type='limit',
            time_in_force='gtc',
            limit_price = current_price * 1.1
        )
    # if an excpetion is thrown, prints exception message and returns 0 to 
    # indicate a failed sell limit placement
    except Exception as e:
        print(e)
        return 0

    # sellLimit function returns 1 if the order is placed succesfully
    else:
        return 1

def sellStocks(stockName, quantity, sellType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='sell',
      type=sellType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )



###########################
###### Bracket Order ######
###########################
def bracketOrder(tick, qty, price):
  return api.submit_order(
    symbol=tick,
    qty=qty,
    side='buy',
    type='market',
    time_in_force='gtc',
    order_class='bracket',
    stop_loss={'stop_price': price * 0.95,
               'limit_price':  price * 0.94},
    take_profit={'limit_price': price * 1.05}
)



#Ben

#####################
#     buyStocks     #
#####################
def buyStocks(stockName, quantity, buyType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='buy',
      type=buyType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )



def limitBuy(stockName, quantity, ToF, price):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='buy',
      type='limit', # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF,#day, gtc, opg, cls, ioc, fok.
      limit_price=price
  )

#Bryan
def trailStopLimit(tick, quantity):
  return api.submit_order(
            symbol=tick,
            qty=quantity,
            side='sell',
            type='trailing_stop',
            trail_percent=1.0,
            time_in_force='gtc'
  )

def getOrders():
  return api.list_orders(
    status='open',
    limit=100,
    nested=False  # show nested multi-leg orders
  )

##############################
# Golden/Death cross checker #
##############################
def maCross(ema, sma, openEprevEMAMA, prevSMA):
  if (ema == sma) and (prevSMA < prevEMA):
    return 1 # golden cross
  elif (ema == sma) and (prevSMA > prevEMA):
    return 2 # death cross
  return 0 # no cross


#determines if the AI should buy right now
def isBuy(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)

  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)

  # check if there is a golden/death cross
  # cross = maCross(ema, sma, prevEMA, prevSMA)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>20) & (curr["rsi"]<50)]
  curr = curr.loc[(curr["macd"]> -.25) & (curr["macd"]< .25)]
  curr = curr.loc[(curr['stoch']> 0) & (curr['stoch']< 30)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty:
    close = curr[ticker]["close"].values
    ema = curr["ema"].values
    if ((close > (ema - (ema*.10))) | (close < (ema*.10) + ema)):
      return True
    #if price not within 10% of ema, return false
    else:
      return False
  #if dataframe is null, return false
  else:
    return False


#determines if the AI should buy right now
def isSell(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)


  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  onvol = OnBalanceVolumeIndicator(close=df[ticker]["close"], volume=df[ticker]["volume"])
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['onbalvol'] = onvol.on_balance_volume()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>60)]
  curr = curr.loc[(curr["macd"]>-1) & (curr["macd"]<.5)]
  curr = curr.loc[(curr['stoch']>60)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty:
    close = curr[ticker]["close"].values
    ema = curr["ema"].values
    if ((close > (ema - ema*.25)) | (close < (ema + ema*.25))):
      sday = df.tail(7)
      head = sday.head(1) 
      tail = sday.tail(1)
      dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      #slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)
      # negative slope returns true, positive returns false
      #if slope < 0:
      #  return True
      #else:
      #  return False
      return True
    #if price not within 25% of ema, return false
    else:
      return False
  #if dataframe is null, return false
  else:
    return False




#################
#LOG REGRESSION ALGO#
#################
def Log_Model(tfone, tftwo, tfthree):
    # Check that the amount of tickers isn't more than 2000
  tickers = getTrendingTickers();
  
  print("The amount of stocks chosen to observe: " + str(len(tickers)))
  # These two lines remove the Stocks folder and then recreate it in order to remove old stocks. Make sure you have created a Stocks Folder the first time you run this.
  shutil.rmtree("./stocks")
  os.mkdir("./stocks")

  # Holds the amount of API calls we executed
  Amount_of_API_Calls = 0

  # This while loop is reponsible for storing the historical data for each ticker in our list. Note that yahoo finance sometimes incurs json.decode errors and because of this we are sleeping for 2
  # seconds after each iteration, also if a call fails we are going to try to execute it again.
  # Also, do not make more than 2,000 calls per hour or 48,000 calls per day or Yahoo Finance may block your IP. The clause "(Amount_of_API_Calls < 1800)" below will stop the loop from making
  # too many calls to the yfinance API.
  # Prepare for this loop to take some time. It is pausing for 2 seconds after importing each stock.

  # Used to make sure we don't waste too many API calls on one Stock ticker that could be having issues
  Stock_Failure = 0
  Stocks_Not_Imported = 0

  # Used to iterate through our list of tickers
  i=0
  while (i < len(tickers)) and (Amount_of_API_Calls < 1800):
      try:
          stock = tickers[i]  # Gets the current stock ticker
          temp = yf.Ticker(str(stock))
          Hist_data = temp.history(period="max")  # Tells yfinance what kind of data we want about this stock (In this example, all of the historical data)
          Hist_data.to_csv("./stocks/"+stock+".csv")  # Saves the historical data in csv format for further processing later
          time.sleep(2)  # Pauses the loop for two seconds so we don't cause issues with Yahoo Finance's backend operations
          Amount_of_API_Calls += 1 
          Stock_Failure = 0
          i += 1  # Iteration to the next ticker
          print("Importing stock data:" + str(i))
      except ValueError:
          print("Yahoo Finance Backend Error, Attempting to Fix")  # An error occured on Yahoo Finance's backend. We will attempt to retreive the data again
          if Stock_Failure > 5:  # Move on to the next ticker if the current ticker fails more than 5 times
              i+=1
              Stocks_Not_Imported += 1
          Amount_of_API_Calls += 1
          Stock_Failure += 1
  print("The amount of stocks we successfully imported: " + str(i - Stocks_Not_Imported))


  # These two lines remove the Stocks folder and then recreate it in order to remove old stocks. Make sure you have created a Stocks Folder the first time you run this.
  # Get the Y values
  list_files = (glob.glob("./stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
  for interval in list_files:
      Stock_Name = ((os.path.basename(interval)).split(".csv")[0])
      data = pd.read_csv(interval)
      dropna(data)
      try: 
        data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume")
      except:
        continue

      data = data.iloc[100:]
      close_prices = data['Close'].tolist()
      firsttf_Obs = []
      secondtf_Obs = []
      thirdtf_Obs = []
      x = 0
      while x < (len(data)):
          if x < (len(data)-tfone):
              if ((close_prices[x+1] + close_prices[x+2] + close_prices[x+3] + close_prices[x+4] + close_prices[x+5])/5) > close_prices[x]:
                  firsttf_Obs.append(1)
              else:
                  firsttf_Obs.append(0)
          else:
              firsttf_Obs.append(0)
          x+=1
      y = 0
      while y < (len(data)):
          if y < (len(data)-tftwo):
              secondtf = 0
              y2 = 0
              while y2 < tftwo:
                  secondtf = secondtf + close_prices[y+y2]
                  y2 += 1
              if (secondtf/tftwo) > close_prices[y]:
                  secondtf_Obs.append(1)
              else:
                  secondtf_Obs.append(0)
          else:
              secondtf_Obs.append(0)
          y+=1
      z = 0
      while z < (len(data)):
          if z < (len(data)-tfthree):
              thirdtf = 0
              z2 = 0
              while z2 < tfthree:
                  thirdtf = thirdtf + close_prices[z+z2]
                  z2 += 1
              if (thirdtf/tfthree) > close_prices[z]:
                  thirdtf_Obs.append(1)
              else:
                  thirdtf_Obs.append(0)
          else:
              thirdtf_Obs.append(0)
          z+=1  
      data['ftf'] = firsttf_Obs
      data['stf'] = secondtf_Obs
      data['ttf'] = thirdtf_Obs
    
      data.to_csv("./stocks/"+Stock_Name+".csv")
      print("Data for " + Stock_Name + " has been substantiated with technical features.")

  Hold_Results = []
  list_files2 = (glob.glob("./stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
  for interval2 in list_files2:
      Stock_Name = ((os.path.basename(interval2)).split(".csv")[0])
      data = pd.read_csv(interval2,index_col=0)
      data = data.replace([np.inf, -np.inf], np.nan)
      data = data.fillna(0)
      try:
        dependents = [data["ftf"].to_list(), data["stf"].to_list(), data["ttf"].to_list()]
  
        data = data.drop(['ftf', 'stf', 'ttf', 'Date', 'Open', 'High', 'Low', 'Close'], axis = 1)
        scaler = StandardScaler()
        data = scaler.fit_transform(data)  # Standardize our data set
        Hold_Results_Section = []
        p = 0
        for dep in dependents:
            x_train, x_test, y_train, y_test =\
            train_test_split(data, dep, test_size=0.2, random_state=0)
            model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr',random_state=0)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)  # To get the predicted values
            conf = confusion_matrix(y_test, y_pred)
            if p == 0:
                Hold_Results.append([Stock_Name, tfone, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
            if p == 1:
                Hold_Results.append([Stock_Name, tftwo, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
            if p == 2:
                Hold_Results.append([Stock_Name, tfthree, model.score(x_train, y_train),model.score(x_test, y_test),conf[0,0],conf[0,1],conf[1,0],conf[1,1]])
            p+=1
        print("Model complete for " + Stock_Name)   
      except:
        continue
  df = pd.DataFrame(Hold_Results, columns=['Stock', 'Observation Period', 'modacctrain', 'modacctest', 'True Positives','False Positives',
  'False Negative','True Negative'])
  df = df[df['modacctest'] > .70]
  getTime = pytz.timezone("America/Chicago")
  getTime = datetime.datetime.now(getTime)
  df.to_csv("./drive/MyDrive/Capstone/Data/" + str(getTime) + ".csv", index = False)
  return df







############################################################################################################################
#####################################            Main Loop           #######################################################
############################################################################################################################
# get trending tickers
account = api.get_account()
loops = 0
strat = 0
clock = api.get_clock()
final = []
while clock.is_open:

  #Get Trail orders
  trail = {}
  #Get Open orders
  orders = getOrders()
  orderList = []
  #Get current holdings
  portfolio = api.list_positions()
  portlist = [] 
  for tick in portfolio:
    portlist.append(tick.symbol) 
  
  #Part of Strat 0 
  if strat == 0:
    for tick in final:
      print("Checking " + tick + " for buy opportunity")
      if tick in portlist:
        continue
      if isBuy(tick, 1000, "minute"):
        stockData = getStockData(tick, 1, "minute").tail(1)
        quantity = (float(account.buying_power)*.1) / stockData[tick]['close'].values
        bracketOrder(tick, int(quantity), float(stockData[tick]['close'].values))
        print("Bought " + str(int(quantity)) + " of " + tick)

    if strat != 0:
      for tick in portlist:
        #Strategy Selection
        if strat == 1:
            if float(tick.unrealized_plpc) > .01:
              if isSell(tick.symbol, 1000, "minute"):
                sellStocks(tick.symbol, int(tick.qty), "market", "day")
                print("Sold: " + tick.symbol)
      print("Portfolio Up to date")
  

  # run ML algo against trending tickets
  # check all entries for all instances where accuracy over 70% for starters
  # returns 7 day stocks, 14 day stocks, 21 day stocks, Observation period column is ints so should be able to compare when buying
  if loops == 0:
    model_Data = Log_Model(7, 14, 21)
    print("log_models Finished")
    initial = model_Data["Stock"].to_list()
    final = []
    for i in initial:
      if i not in final:
        final.append(i)
  # run isbuy for all watch list tickers on specified timelength where accuracy over 75%
  # buy if true returned
    for tick in final:
      print("Checking: " + tick)
      if isBuy(tick, 1000 , "minute"):
        tData = getStockData(tick, 1, "minute").tail(1)
        quantity = (float(account.buying_power)*.1) / float(tData[tick]['close'].values)
        if int(quantity) == 0:
          continue
        if tick in portlist:
          continue
        bracketOrder(tick, int(quantity), float(tData[tick]['close'].values))
        #buyStocks(tick, int(quantity), "market","day")
        print("Bought " + str(int(quantity)) + " of " + str(tick))
        portlist.append(str(tick))
  if loops >= 5:
    loops = 0
  else:
    loops += 1

  time.sleep(60)

ModuleNotFoundError: ignored